# Gradient Boosted Trees

In [106]:
from fastai.tabular.all import *
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer

from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble import HistGradientBoostingClassifier

path = Path('/Users/baranserajelahi/Codes/fraud-detection-pytorch-scikit-fastai/data')
Path.BASE_PATH = path
pd.options.display.max_rows = 200
pd.options.display.max_columns = 120

In [107]:
# Tabular object created and saved on the 02_ notebook
to = (path/'to.pkl').load()

In [108]:
xs, y = to.train.xs, to.train.y
valid_xs, valid_y = to.valid.xs, to.valid.y

## HistGradientBoostingClassifier

In [112]:
'Previously, leaf-wise growth was an exclusive feature of lightGBM, but xgboost has since implemented this growth strategy (this change has not been reflected in the lightGBM docs, but has been acknowledged in a blog post). This strategy is only available for the histogram-based method (which I will explain below), so in order to use it, users will have to set the tree_method parameter to hist and the grow_policy parameter to lossguide.'

'Previously, leaf-wise growth was an exclusive feature of lightGBM, but xgboost has since implemented this growth strategy (this change has not been reflected in the lightGBM docs, but has been acknowledged in a blog post). This strategy is only available for the histogram-based method (which I will explain below), so in order to use it, users will have to set the tree_method parameter to hist and the grow_policy parameter to lossguide.'

In [113]:
'Xgboost offers the option tree_method=approx, which computes a new set of bins at each split using the gradient statistics. LightGBM and xgboost with the tree_method set to hist will both compute the bins at the beginning of training and reuse the same bins throughout the entire training process.'

'Xgboost offers the option tree_method=approx, which computes a new set of bins at each split using the gradient statistics. LightGBM and xgboost with the tree_method set to hist will both compute the bins at the beginning of training and reuse the same bins throughout the entire training process.'

In [114]:
'BOTH: Though lightGBM does not enable ignoring zero values by default, it has an option called zero_as_missing which, if set to True, will regard all zero values as missing. According to this thread on GitHub, lightGBM will treat missing values in the same way as xgboost as long as the parameter use_missing is set to True (which is the default behavior).'

'Though lightGBM does not enable ignoring zero values by default, it has an option called zero_as_missing which, if set to True, will regard all zero values as missing. According to this thread on GitHub, lightGBM will treat missing values in the same way as xgboost as long as the parameter use_missing is set to True (which is the default behavior).'

In [115]:
'LGBM: This is a method introduced in lightGBM that also takes advantage of the sparsity of large datasets. The essential observation behind this method is that the sparsity of features means that some features are never non-zero together. For instance, the words "Python" and "protein" might never appear in the same document in the data. This means that these features can be "bundled" into a single feature without losing any information. Suppose the tf-idf score for "Python" ranges from 0 to 10 and the tf-idf score for  "protein" ranges from 0 to 20. In this case, the feature'

'This is a method introduced in lightGBM that also takes advantage of the sparsity of large datasets. The essential observation behind this method is that the sparsity of features means that some features are never non-zero together. For instance, the words "Python" and "protein" might never appear in the same document in the data. This means that these features can be "bundled" into a single feature without losing any information. Suppose the tf-idf score for "Python" ranges from 0 to 10 and the tf-idf score for  "protein" ranges from 0 to 20. In this case, the feature'

In [116]:
'LGBM: Unfortunately, the problem of finding the most efficient bundle is NP-hard. Therefore, the authors of the paper opted for an approximate algorithm that tolerated a certain degree of overlap \gamma  between the non-zero elements within a feature bundle. The details of this algorithm are beyond the scope of this post, so please refer to the original paper for details.'

'Unfortunately, the problem of finding the most efficient bundle is NP-hard. Therefore, the authors of the paper opted for an approximate algorithm that tolerated a certain degree of overlap \\gamma  between the non-zero elements within a feature bundle. The details of this algorithm are beyond the scope of this post, so please refer to the original paper for details.'

HistGradientBoostingClassifier is based on Microsofts LightGBM. |

### Default parameters 

In [109]:
def m_roc(y, xs):
    preds = clf.predict_proba(xs)
    preds_isFraud = preds[:,1]
    return roc_auc_score(y, preds_isFraud)

The max_iter parameter was increased to 1000 from its default value of 100.

In [110]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1,
                                    max_iter=1000, scoring='roc_auc')
clf.fit(xs,y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.042 GB of training data: 0.269 s
Binning 0.005 GB of validation data: 0.018 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 31 leaves, max depth = 8, train score: 0.84080, val score: 0.83438, in 0.066s
[2/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.86431, val score: 0.85824, in 0.069s
[3/1000] 1 tree, 31 leaves, max depth = 8, train score: 0.87042, val score: 0.86006, in 0.050s
[4/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.87092, val score: 0.85956, in 0.046s
[5/1000] 1 tree, 31 leaves, max depth = 13, train score: 0.87622, val score: 0.86071, in 0.056s
[6/1000] 1 tree, 31 leaves, max depth = 8, train score: 0.88018, val score: 0.86098, in 0.061s
[7/1000] 1 tree, 31 leaves, max depth = 8, train score: 0.88209, val score: 0.86304, in 0.047s
[8/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.88800, val score: 0.86607, in 0.045s
[9/1000] 1 tree, 31 leaves, max depth = 12, train score: 0.89025, val score: 0.86711, in 0.049s
[10/1000] 1 tree, 31 

(0.9721576934960269, 0.8553771096898483)

The algorithm may have stopped too early, so let's increase the threshold for early stopping by decreasing tol by a factor of 10 to e-8 from its default value e-7.

In [111]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1,
                                     max_iter=1000, scoring='roc_auc', tol=1e-8)
clf.fit(xs,y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.042 GB of training data: 0.279 s
Binning 0.005 GB of validation data: 0.018 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 31 leaves, max depth = 8, train score: 0.82468, val score: 0.82268, in 0.044s
[2/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.83815, val score: 0.84384, in 0.045s
[3/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.84699, val score: 0.85118, in 0.046s
[4/1000] 1 tree, 31 leaves, max depth = 12, train score: 0.84930, val score: 0.85342, in 0.046s
[5/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.85351, val score: 0.85865, in 0.047s
[6/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.85587, val score: 0.86454, in 0.052s
[7/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.85553, val score: 0.86417, in 0.047s
[8/1000] 1 tree, 31 leaves, max depth = 13, train score: 0.86260, val score: 0.86365, in 0.050s
[9/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.86249, val score: 0.86497, in 0.051s
[10/1000] 1 tree, 3

[85/1000] 1 tree, 31 leaves, max depth = 12, train score: 0.98203, val score: 0.91004, in 0.106s
[86/1000] 1 tree, 31 leaves, max depth = 12, train score: 0.98229, val score: 0.90967, in 0.073s
[87/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.98223, val score: 0.91031, in 0.069s
[88/1000] 1 tree, 31 leaves, max depth = 17, train score: 0.98305, val score: 0.91030, in 0.069s
[89/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.98335, val score: 0.91228, in 0.069s
[90/1000] 1 tree, 31 leaves, max depth = 14, train score: 0.98344, val score: 0.91249, in 0.071s
[91/1000] 1 tree, 31 leaves, max depth = 12, train score: 0.98410, val score: 0.91291, in 0.067s
[92/1000] 1 tree, 31 leaves, max depth = 12, train score: 0.98448, val score: 0.91216, in 0.068s
[93/1000] 1 tree, 31 leaves, max depth = 15, train score: 0.98583, val score: 0.91212, in 0.068s
[94/1000] 1 tree, 31 leaves, max depth = 12, train score: 0.98602, val score: 0.91189, in 0.072s
[95/1000] 1 tree, 31 leaves, max

(0.9813462443575784, 0.8539935554339263)

### Hyperparameter tuning

Hyperparameters were tuned interatively. Using both GridSearchCV and RandomizedGridSearchCV. RandomizedGridSearchCV was used for all 5 sets of parameters, than the final parameter set was double checked using GridSearchCV. Only the final results are displayed in the notenbook.

In [30]:
parameters1 = {
 'max_iter': [1000],
 'learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3],
 'max_depth': [None, 25, 50, 75],
 'l2_regularization': [0, 1.5],
 'max_bins': [255],
 'max_leaf_nodes': [20, 31, 40, 50, 60],
 'min_samples_leaf': [1, 5, 10, 20],
 'random_state': [33],
 }

In [7]:
parameters2 = {
 'max_iter': [1000],
 'learning_rate': [0.01, 0.1, 0.2, 0.3],
 'max_depth': [None, 25, 50],
 'l2_regularization': [0, 1.5],
 'max_bins': [255],
 'max_leaf_nodes': [20, 31, 40],
 'min_samples_leaf': [1, 5],
 'random_state': [33],
 }

In [23]:
parameters3 = {
 'max_iter': [1000],
 'learning_rate': [0.005, 0.01, 0.1],
 'max_depth': [15, 25, 35],
 'l2_regularization': [1.5, 2.0],
 'max_bins': [255],
 'max_leaf_nodes': [40, 45, 50],
 'min_samples_leaf': [5, 7, 10],
 'random_state': [33],
 }

In [31]:
parameters4 = {
 'max_iter': [1000],
 'learning_rate': [0.01, 0.02],
 'max_depth': [25, 30],
 'l2_regularization': [2.0, 2.5, 3.0],
 'max_bins': [255],
 'max_leaf_nodes': [37, 40, 43],
 'min_samples_leaf': [7, 8, 9],
 'random_state': [33],
 }

In [36]:
parameters5 = {
 'max_iter': [1000],
 'learning_rate': [0.01, 0.02, 0.03],
 'max_depth': [25, 27],
 'l2_regularization': [2.4, 2.5, 2.6],
 'max_bins': [255],
 'max_leaf_nodes': [43, 44, 45],
 'random_state': [33],
 }

#### GridSearchCV

In [37]:
clf = GridSearchCV(HistGradientBoostingClassifier(loss='binary_crossentropy', max_iter=1000, 
                                                  scoring='roc_auc', tol=1e-8 ), parameters5, 
                   scoring=make_scorer(roc_auc_score, needs_proba=True), n_jobs=-1)

In [38]:
%%time
clf.fit(xs, y)
clf.best_estimator_, clf.best_score_, clf.best_params_

CPU times: user 1min 24s, sys: 22.1 s, total: 1min 46s
Wall time: 11min 42s


(HistGradientBoostingClassifier(l2_regularization=2.4, learning_rate=0.03,
                                loss='binary_crossentropy', max_depth=25,
                                max_iter=1000, max_leaf_nodes=44,
                                random_state=33, scoring='roc_auc', tol=1e-08),
 0.8691008607988312,
 {'l2_regularization': 2.4,
  'learning_rate': 0.03,
  'max_bins': 255,
  'max_depth': 25,
  'max_iter': 1000,
  'max_leaf_nodes': 44,
  'random_state': 33})

In [42]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1, l2_regularization=2.4,
                             learning_rate=0.03, max_depth=25, max_iter=1000, max_leaf_nodes=44,
                             min_samples_leaf=8,  scoring='roc_auc', tol=1e-8)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.042 GB of training data: 0.273 s
Binning 0.005 GB of validation data: 0.019 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.83422, val score: 0.83526, in 0.076s
[2/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.84166, val score: 0.84216, in 0.081s
[3/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.84365, val score: 0.84422, in 0.081s
[4/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.84635, val score: 0.84409, in 0.120s
[5/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.85918, val score: 0.85764, in 0.079s
[6/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.85980, val score: 0.85770, in 0.122s
[7/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.85984, val score: 0.85860, in 0.120s
[8/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.86009, val score: 0.85848, in 0.130s
[9/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.86031, val score: 0.85919, in 0.116s
[10/1000] 1 tree,

[85/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.92742, val score: 0.90149, in 0.116s
[86/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.92801, val score: 0.90142, in 0.117s
[87/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.92856, val score: 0.90216, in 0.111s
[88/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.92926, val score: 0.90224, in 0.111s
[89/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.92989, val score: 0.90258, in 0.107s
[90/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.93051, val score: 0.90300, in 0.181s
[91/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.93145, val score: 0.90335, in 0.106s
[92/1000] 1 tree, 44 leaves, max depth = 19, train score: 0.93208, val score: 0.90435, in 0.272s
[93/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.93265, val score: 0.90504, in 0.110s
[94/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.93347, val score: 0.90482, in 0.107s
[95/1000] 1 tree, 44 leaves, m

[169/1000] 1 tree, 44 leaves, max depth = 20, train score: 0.96254, val score: 0.91969, in 0.204s
[170/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.96271, val score: 0.91994, in 0.156s
[171/1000] 1 tree, 44 leaves, max depth = 18, train score: 0.96299, val score: 0.91994, in 0.145s
[172/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.96309, val score: 0.92024, in 0.142s
[173/1000] 1 tree, 44 leaves, max depth = 18, train score: 0.96335, val score: 0.92045, in 0.171s
[174/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.96377, val score: 0.92087, in 0.175s
[175/1000] 1 tree, 44 leaves, max depth = 16, train score: 0.96408, val score: 0.92078, in 0.150s
[176/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.96426, val score: 0.92085, in 0.141s
[177/1000] 1 tree, 44 leaves, max depth = 21, train score: 0.96446, val score: 0.92099, in 0.145s
[178/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.96457, val score: 0.92096, in 0.211s
[179/1000] 1 tree, 4

(0.9659117013340929, 0.8577956250355605)

#### RandomizedSearchCV

In [43]:
clf = RandomizedSearchCV(HistGradientBoostingClassifier(loss='binary_crossentropy', max_iter=1000, 
                                                  scoring='roc_auc', tol=1e-8 ), parameters5, 
                   scoring=make_scorer(roc_auc_score, needs_proba=True), n_jobs=-1)

In [44]:
%%time
clf.fit(xs, y)
clf.best_estimator_, clf.best_score_, clf.best_params_

CPU times: user 1min 19s, sys: 18 s, total: 1min 37s
Wall time: 2min 17s


(HistGradientBoostingClassifier(l2_regularization=2.4, learning_rate=0.03,
                                loss='binary_crossentropy', max_depth=27,
                                max_iter=1000, max_leaf_nodes=44,
                                random_state=33, scoring='roc_auc', tol=1e-08),
 0.8691008607988312,
 {'random_state': 33,
  'max_leaf_nodes': 44,
  'max_iter': 1000,
  'max_depth': 27,
  'max_bins': 255,
  'learning_rate': 0.03,
  'l2_regularization': 2.4})

In [45]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1, l2_regularization=2.4,
                             learning_rate=0.03, max_depth=27, max_iter=1000, max_leaf_nodes=44,
                             min_samples_leaf=8,  scoring='roc_auc', tol=1e-8)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.042 GB of training data: 0.259 s
Binning 0.005 GB of validation data: 0.019 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.82760, val score: 0.80686, in 0.052s
[2/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.83003, val score: 0.80806, in 0.101s
[3/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.83584, val score: 0.81004, in 0.050s
[4/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.84916, val score: 0.83466, in 0.052s
[5/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.85368, val score: 0.83562, in 0.053s
[6/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.85301, val score: 0.83424, in 0.053s
[7/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.85311, val score: 0.83412, in 0.051s
[8/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.85305, val score: 0.83381, in 0.055s
[9/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.85380, val score: 0.83552, in 0.052s
[10/1000] 1 tre

[85/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.92146, val score: 0.87816, in 0.105s
[86/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.92232, val score: 0.87912, in 0.083s
[87/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.92255, val score: 0.87928, in 0.080s
[88/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.92288, val score: 0.87950, in 0.074s
[89/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.92380, val score: 0.88020, in 0.117s
[90/1000] 1 tree, 44 leaves, max depth = 19, train score: 0.92415, val score: 0.88021, in 0.065s
[91/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.92469, val score: 0.88100, in 0.067s
[92/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.92538, val score: 0.88118, in 0.077s
[93/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.92579, val score: 0.88122, in 0.076s
[94/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.92619, val score: 0.88132, in 0.080s
[95/1000] 1 tree, 44 leaves, m

[252/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.97205, val score: 0.90193, in 0.132s
[253/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.97217, val score: 0.90204, in 0.124s
[254/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.97223, val score: 0.90219, in 0.130s
[255/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.97230, val score: 0.90210, in 0.132s
[256/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.97251, val score: 0.90237, in 0.113s
[257/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.97260, val score: 0.90257, in 0.129s
[258/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.97271, val score: 0.90271, in 0.133s
[259/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.97284, val score: 0.90277, in 0.147s
[260/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.97300, val score: 0.90276, in 0.142s
[261/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.97324, val score: 0.90265, in 0.133s
[262/1000] 1 tree, 4

[336/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.98139, val score: 0.90867, in 0.190s
[337/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.98145, val score: 0.90875, in 0.158s
[338/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.98155, val score: 0.90877, in 0.295s
[339/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.98161, val score: 0.90901, in 0.219s
[340/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.98163, val score: 0.90904, in 0.152s
[341/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.98172, val score: 0.90914, in 0.205s
[342/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.98180, val score: 0.90921, in 0.250s
[343/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.98187, val score: 0.90927, in 0.197s
[344/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.98190, val score: 0.90934, in 0.150s
[345/1000] 1 tree, 44 leaves, max depth = 18, train score: 0.98194, val score: 0.90940, in 0.151s
[346/1000] 1 tree, 4

(0.9795618885673211, 0.865225601216363)

#### Including the V*** columns

In [76]:
to = (path/'to2.pkl').load()

xs, y             = to.train.xs, to.train.y
valid_xs, valid_y = to.valid.xs, to.valid.y

In [77]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1, l2_regularization=2.4,
                             learning_rate=0.03, max_depth=27, max_iter=1000, max_leaf_nodes=44,
                             min_samples_leaf=8,  scoring='roc_auc', tol=1e-8)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.279 GB of training data: 1.767 s
Binning 0.031 GB of validation data: 0.045 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.85386, val score: 0.84830, in 0.355s
[2/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.86144, val score: 0.84919, in 0.407s
[3/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.86312, val score: 0.85426, in 0.451s
[4/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.86442, val score: 0.85297, in 0.446s
[5/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.86672, val score: 0.85341, in 0.284s
[6/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.86692, val score: 0.85281, in 0.275s
[7/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.87052, val score: 0.85437, in 0.460s
[8/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.87125, val score: 0.85707, in 0.392s
[9/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.87168, val score: 0.85724, in 0.631s
[10/1000] 1 tree,

[85/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.93397, val score: 0.89907, in 0.288s
[86/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.93462, val score: 0.89929, in 1.158s
[87/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.93534, val score: 0.89986, in 0.291s
[88/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.93573, val score: 0.89958, in 0.288s
[89/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.93614, val score: 0.89977, in 0.303s
[90/1000] 1 tree, 44 leaves, max depth = 16, train score: 0.93715, val score: 0.90008, in 0.309s
[91/1000] 1 tree, 44 leaves, max depth = 18, train score: 0.93734, val score: 0.90057, in 0.321s
[92/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.93795, val score: 0.90069, in 0.321s
[93/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.93839, val score: 0.90050, in 0.318s
[94/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.93868, val score: 0.90042, in 0.336s
[95/1000] 1 tree, 44 leaves, m

(0.9458704971059383, 0.8629394444253007)

## GradientBoostingClassifier 

HistGradientBoostingClassifier is recommended for datasets with a large number of classes. Since this is a binary classification problem, I decided to try the more established GradientBoostingClassifier class also.

In [49]:
from sklearn.ensemble import GradientBoostingClassifier 

In [117]:
GradientBoostingClassifier?

In [58]:
# Tabular object created and saved on the 02_ notebook
to = (path/'to.pkl').load()

xs, y = to.train.xs, to.train.y
valid_xs, valid_y = to.valid.xs, to.valid.y

### Defaults

n_estimators was increased to 1000 from its default value of 100.

In [61]:
clf = GradientBoostingClassifier(loss='deviance', n_estimators=1000, verbose=1)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

      Iter       Train Loss   Remaining Time 
         1           0.2858            3.30m
         2           0.2787            3.31m
         3           0.2732            3.41m
         4           0.2681            3.42m
         5           0.2632            3.47m
         6           0.2587            3.47m
         7           0.2550            3.47m
         8           0.2521            3.47m
         9           0.2490            3.47m
        10           0.2456            3.46m
        20           0.2276            3.40m
        30           0.2183            3.34m
        40           0.2126            3.27m
        50           0.2085            3.23m
        60           0.2047            3.22m
        70           0.2017            3.20m
        80           0.1991            3.17m
        90           0.1969            3.15m
       100           0.1948            3.12m
       200           0.1771            2.83m
       300           0.1646            2.52m
       40

(0.9705755015972327, 0.8568501495478038)

### Hyperparameter Tuning

In [63]:
parameters1 = {
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'n_estimators': [40, 50, 60, 70, 1000]
}

In [72]:
parameters2 = {
 'max_depth': [3, 6, 9, 12],
 'subsample': [0.8, 0.66],
 'min_samples_split': [2, 5, 10],
 'max_leaf_nodes': [None, 20, 31, 40],
 'min_samples_leaf': [1, 5],
 'random_state': [33],
 }

In [78]:
parameters3 = {
 'max_depth': [8, 9, 10],
 'subsample': [0.60, 0.66, 0.72],
 'min_samples_split': [3, 5, 8],
 'max_leaf_nodes': [None, 45, 50, 60],
 'min_samples_leaf': [5, 7, 9],
 'random_state': [33],
}

In [81]:
parameters4 = {
 'max_depth': [8, 9, 10],
 'subsample': [0.68, 0.72, 0.76],
 'min_samples_split': [1, 2, 3],
 'max_leaf_nodes': [50, 52, 54],
 'min_samples_leaf': [6, 7, 8],
 'random_state': [33],
 }

In [99]:
parameters5 = {
    'n_estimators': [40, 50, 60, 70, 100, 1000]
}

In [100]:
clf = GridSearchCV(GradientBoostingClassifier(loss='deviance', learning_rate=0.1, 
                         criterion='friedman_mse', tol=1e-8, max_features='sqrt',
                                subsample=0.72, min_samples_split=6, 
                                max_leaf_nodes=52, max_depth=10), 
                         parameters5, 
                   scoring=make_scorer(roc_auc_score, needs_proba=True), n_jobs=-1)

In [101]:
%%time
clf.fit(xs, y)
clf.best_estimator_, clf.best_score_, clf.best_params_

CPU times: user 1min 12s, sys: 1.32 s, total: 1min 13s
Wall time: 4min 14s


(GradientBoostingClassifier(max_depth=10, max_features='sqrt', max_leaf_nodes=52,
                            min_samples_split=6, n_estimators=1000,
                            subsample=0.72, tol=1e-08),
 0.8841218032816766,
 {'n_estimators': 1000})

In [104]:
clf = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, verbose=1,
                         criterion='friedman_mse', tol=1e-8, max_features='sqrt',
                                subsample=0.72, min_samples_split=6, 
                                max_leaf_nodes=52, max_depth=10, n_estimators=1000)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.2693           0.0407            1.12m
         2           0.2610           0.0080            1.04m
         3           0.2520           0.0075           59.71s
         4           0.2429           0.0055            1.08m
         5           0.2422           0.0028            1.06m
         6           0.2346           0.0033            1.05m
         7           0.2353           0.0050            1.04m
         8           0.2226           0.0042            1.03m
         9           0.2224           0.0027            1.03m
        10           0.2239           0.0020            1.04m
        20           0.1882           0.0006            1.07m
        30           0.1758           0.0002            1.03m
        40           0.1648          -0.0000            1.03m
        50           0.1593          -0.0001            1.03m
        60           0.1503          -0.0010            1.02m
       

(1.0, 0.8318749851413832)

In [103]:
clf = GradientBoostingClassifier(loss='deviance', learning_rate=0.005, verbose=1,
                         criterion='friedman_mse', tol=1e-8, max_features='sqrt',
                                subsample=0.72, min_samples_split=6, 
                                max_leaf_nodes=52, max_depth=10, n_estimators=1000)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.3119           0.0022            1.08m
         2           0.3162           0.0018            1.10m
         3           0.3054           0.0018            1.03m
         4           0.3067           0.0014            1.06m
         5           0.3047           0.0014            1.05m
         6           0.3057           0.0015            1.05m
         7           0.2964           0.0015            1.06m
         8           0.3050           0.0014            1.05m
         9           0.2936           0.0013            1.06m
        10           0.2941           0.0012            1.02m
        20           0.2828           0.0009           58.00s
        30           0.2784           0.0006           58.51s
        40           0.2696           0.0005           58.59s
        50           0.2593           0.0004           57.99s
        60           0.2554           0.0004           58.28s
       

(0.9152136625826836, 0.843631924082018)

In [105]:
clf = GradientBoostingClassifier(loss='deviance', learning_rate=0.01, verbose=1,
                         criterion='friedman_mse', tol=1e-8, max_features='sqrt',
                                subsample=0.72, min_samples_split=6, 
                                max_leaf_nodes=52, max_depth=10, n_estimators=1000)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.3082           0.0047            1.15m
         2           0.3072           0.0027            1.30m
         3           0.3074           0.0027            1.21m
         4           0.2995           0.0030            1.21m
         5           0.3000           0.0023            1.16m
         6           0.2976           0.0022            1.14m
         7           0.2994           0.0018            1.18m
         8           0.2937           0.0017            1.17m
         9           0.2912           0.0017            1.17m
        10           0.2880           0.0020            1.19m
        20           0.2695           0.0011            1.27m
        30           0.2599           0.0008            1.23m
        40           0.2517           0.0007            1.22m
        50           0.2427           0.0005            1.20m
        60           0.2351           0.0004            1.17m
       

(0.9472911054564462, 0.8462935197198218)